In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, models, layers, utils, activations, losses, optimizers, metrics
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#import tensorflow_datasets as tfd 載入失敗 下載應該就掛了
from glob import glob
from tqdm import tqdm
import cv2
import os 
import random
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from tensorflow.keras.applications import *
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

In [2]:
#兩類 dog & cat
num_classes = 2
IMG_SIZE =  128

In [3]:
paths_cat = glob('PetImages/Cat3/*.jpg')
paths_dog = glob('PetImages/Dog3/*.jpg')

In [4]:
all_path = paths_cat + paths_dog
random.shuffle(all_path) #打亂排序

In [5]:
def create_dataset(paths, img_size=IMG_SIZE, num_classes=num_classes, test=False):
    num_data = len(paths)
    x_data = np.empty((num_data, img_size, img_size, 3))
    y_data = np.empty((num_data))
    
    for i,path in enumerate(tqdm(paths)):
        try:
            img = cv2.imread(path)[:,:,::-1]
            img = cv2.resize(img, (img_size, img_size))
            img = img/255.
            x_data[i] = img

            if test is False:
                filename = os.path.split(path)[0]
                if filename == 'PetImages/Cat3':
                    y_data[i] = 0
                else:
                    y_data[i] = 1
                    
        except:
            #print(path)
            continue
    if not test:
            y_data = utils.to_categorical(y_data, num_classes=num_classes)
            return x_data, y_data
    else:
        return x_data

In [6]:
x_data, y_data = create_dataset(all_path)

100%|███████████████████████████████████████████████████████████████████████████| 11437/11437 [00:39<00:00, 289.85it/s]


In [7]:
#split dataset
x_data, x_test, y_data, y_test  = train_test_split(x_data,y_data, test_size=0.1, random_state=0)
x_train, x_val, y_train, y_val  = train_test_split(x_data,y_data, test_size=0.1, random_state=0)


In [ ]:
fold_no = 1
for train, test in kfold.split(inputs, targets):
    

In [ ]:
inputs = np.concatenate((x_train, y_train), axis=0)
targets = np.concatenate((x_val, y_val), axis=0)

In [8]:
# create model
model = models.Sequential([layers.Input((IMG_SIZE, IMG_SIZE, 3)),
                           layers.Conv2D(filters=4, kernel_size=5, activation='relu'),    
                           layers.MaxPool2D(2),
                           layers.Conv2D(filters=8, kernel_size=3, activation='relu'),        
                           layers.MaxPool2D(2),
                           layers.Conv2D(filters=16, kernel_size=3, activation='relu'),
                           layers.MaxPool2D(2),
                           layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
                           layers.MaxPool2D(2),
                           layers.Flatten(),
                           layers.Dense(128, activation='relu'),
                           layers.Dense(num_classes, activation='sigmoid'),
])
# Compile model
model.compile(loss=losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(x_data, y_data, validation_split=0.1, epochs=10, batch_size=64)


Epoch 1/10
145/145 [==============================] - 11s 22ms/step - loss: 0.6734 - accuracy: 0.5762 - val_loss: 0.6242 - val_accuracy: 0.6592
Epoch 2/10
145/145 [==============================] - 2s 15ms/step - loss: 0.5932 - accuracy: 0.6857 - val_loss: 0.5814 - val_accuracy: 0.6913
Epoch 3/10
145/145 [==============================] - 2s 15ms/step - loss: 0.5380 - accuracy: 0.7352 - val_loss: 0.5469 - val_accuracy: 0.7233
Epoch 4/10
145/145 [==============================] - 2s 15ms/step - loss: 0.4958 - accuracy: 0.7626 - val_loss: 0.5205 - val_accuracy: 0.7408
Epoch 5/10
145/145 [==============================] - 2s 15ms/step - loss: 0.4625 - accuracy: 0.7851 - val_loss: 0.5213 - val_accuracy: 0.7573
Epoch 6/10
145/145 [==============================] - 2s 15ms/step - loss: 0.4372 - accuracy: 0.7978 - val_loss: 0.5001 - val_accuracy: 0.7641
Epoch 7/10
145/145 [==============================] - 2s 15ms/step - loss: 0.4163 - accuracy: 0.8072 - val_loss: 0.4998 - val_accuracy: 0.760

In [9]:
# define 10-fold cross validation test harness
seed = 7
np.random.seed(seed)

# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []

In [9]:
kf = KFold(n_splits=10)
cvscores = []

for train_index, val_index in kf.split(x_data,y_data):
#     print('train_index', train_index, 'test_index', val_index)
    train_x, train_y = x_data[train_index], y_data[train_index]
    val_x, val_y = x_data[val_index], y_data[val_index]
    # create model
    model = models.Sequential([layers.Input((IMG_SIZE, IMG_SIZE, 3)),
                               layers.Conv2D(filters=4, kernel_size=5, activation='relu'),    
                               layers.MaxPool2D(2),
                               layers.Conv2D(filters=8, kernel_size=3, activation='relu'),        
                               layers.MaxPool2D(2),
                               layers.Conv2D(filters=16, kernel_size=3, activation='relu'),
                               layers.MaxPool2D(2),
                               layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
                               layers.MaxPool2D(2),
                               layers.Flatten(),
                               layers.Dense(128, activation='relu'),
                               layers.Dense(num_classes, activation='sigmoid'),
    ])
    # Compile model
    model.compile(loss=losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
    # Fit the model
    model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=10, batch_size=64)
    scores = model.evaluate(val_x[val_index], val_y[val_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Epoch 1/10
142/145 [============================>.] - ETA: 0s - loss: 0.6617 - accuracy: 0.5926

ResourceExhaustedError: Graph execution error:

OOM when allocating tensor with shape[64,3,128,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node sequential_1/conv2d_4/Conv2D-0-TransposeNHWCToNCHW-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_test_function_9661]

In [ ]:
for i in range(0,10):
    x_data,y_data = 

In [11]:
for train, val in kfold.split(x_data, y_data):
  # create model
    model = models.Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss=losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
    # Fit the model
    model.fit(x_data[train], y_data[train], epochs=150, batch_size=10, verbose=0)
    # evaluate the model
    scores = model.evaluate(X[test], Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

ValueError: Supported target types are: ('binary', 'multiclass'). Got 'multilabel-indicator' instead.